In [1]:
import os
import numpy as np
from nilearn import image
from nilearn import plotting
import nibabel as nib

import matplotlib.pyplot as plt
import pandas as pd



In [ ]:
#load subject data downloaded from local computer and uploaded onto notebook 
img = nib.load('sub-10274_task-bart_bold_space-MNI152NLin2009cAsym_preproc.nii')

In [ ]:
img_1 = image.index_img(img,1)
plotting.view_img(img_1, threshold=None, bg_img=False)

In [ ]:
img.shape

In [ ]:
from nilearn import datasets

msdl_atlas = datasets.fetch_atlas_msdl()

msdl_coords = msdl_atlas.region_coords
n_regions = len(msdl_coords)

print(f'MSDL has {n_regions} ROIs, part of the following networks :\n{np.unique(msdl_atlas.networks)}.')

In [ ]:
plotting.plot_prob_atlas(msdl_atlas.maps)

In [ ]:
from nilearn import maskers

In [ ]:
masker = maskers.NiftiMapsMasker(
    msdl_atlas.maps, resampling_target="data",
    t_r=2, detrend=True,
    low_pass=0.1, high_pass=0.01).fit()

In [ ]:
roi_time_series = masker.transform(img)
roi_time_series.shape

In [ ]:
#creating connnectome -- functional connectivity (pairwise correlations between ROIs)

from nilearn.connectome import ConnectivityMeasure

correlation_measure = ConnectivityMeasure(kind='correlation')
correlation_matrix = correlation_measure.fit_transform([roi_time_series])[0]

In [ ]:
np.fill_diagonal(correlation_matrix, 0)
plotting.plot_matrix(correlation_matrix, labels=msdl_atlas.labels,
                     vmax=0.8, vmin=-0.8, colorbar=True)

#can see functional connectivity as defined by correlation by all 39 ROIs 

In [ ]:
#can view as embedded connectome 

plotting.view_connectome(correlation_matrix, edge_threshold=0.2,
                         node_coords=msdl_atlas.region_coords)

In [ ]:
# We threshold to keep only the 20% of edges with the highest value
# because the graph is very dense
plotting.plot_connectome(
    correlation_matrix, msdl_coords, edge_threshold="80%", colorbar=True
)

plotting.show()

In [ ]:
#need functional connectivity data and labels and then use them
#as the inputs to train DNN below

#then use pairwise comparison between tasks of interest 
#(e.g., risky decision making via BART vs. response inhibition via SST)
#goal: can we classify task comparison with high accuracy by using a DNN?
#if we can find high classification accuracy with decoded features considered
#highly salient by decoder, this could potentially show correlations between diff
#networks that may be commonly known for their activation during task performance


#then we reshape the correlation matrix into a flat vector, but when we plot
#we can reshape it to original shape

In [ ]:
#Fine-Tuning and Optimization:
#Depending on the performance of your initial model, 
#you might need to fine-tune various hyperparameters, 
#adjust the network architecture, or use regularization techniques to prevent overfitting.



In [ ]:
#next need to get feature salient scores
#install deeplift library in terminal
#used pip install deeplift 

#from deeplift.layers import NonlinearMxtsMode
#from deeplift.conversion import kerasapi_conversion as kc
#from deeplift.blobs import NonlinearMxtsMode

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers, models
from keras.datasets import fashion_mnist

import deeplift
import numpy as np
import keras
from keras.datasets import fashion_mnist
from keras.models import load_model
from deeplift.layers import NonlinearMxtsMode
from deeplift.conversion import kerasapi_conversion as kc

import h5py
import deepexplain
import matplotlib
import skimage
import scipy
import numpy as np
from keras.models import load_model

In [3]:
# Import DeepExplain to draw saliency maps -- compatibility issue between TensorFlow versions 
#and DeepExplain library so had to bypass it by adding a line of code in DeepExplain library on 
#jupyter notebook in home dir. asked terminal to reinstall it using updated code and restarted
#kernel and worked afterwards
from deepexplain.tensorflow import DeepExplain

In [5]:
# Load and preprocess the Fashion MNIST dataset
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [6]:
# Build the DNN model
keras_model = models.Sequential([
    layers.Flatten(input_shape=(28, 28)),  # Flatten the 28x28 images to a 1D vector
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')  # 10 classes for the different clothing items
])

In [7]:
# Compile the model
keras_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
# Train the model
history = keras_model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10


2023-08-18 19:04:41.951999: W tensorflow/c/c_api.cc:291] Operation '{name:'dense/kernel/Assign' id:14 op device:{requested: '', assigned: ''} def:{{{node dense/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense/kernel, dense/kernel/Initializer/random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


59776/60000 [============================>.] - ETA: 0s - loss: 0.5023 - acc: 0.8214

/srv/conda/envs/notebook/lib/python3.10/site-packages/keras/engine/training_v1.py:2333: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates
2023-08-18 19:04:45.442895: W tensorflow/c/c_api.cc:291] Operation '{name:'loss/mul' id:125 op device:{requested: '', assigned: ''} def:{{{node loss/mul}} = Mul[T=DT_FLOAT, _has_manual_control_dependencies=true](loss/mul/x, loss/dense_2_loss/value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


60000/60000 [==============================] - 4s 63us/sample - loss: 0.5018 - acc: 0.8216 - val_loss: 0.4139 - val_acc: 0.8503
Epoch 2/10
60000/60000 [==============================] - 3s 54us/sample - loss: 0.3723 - acc: 0.8648 - val_loss: 0.3905 - val_acc: 0.8577
Epoch 3/10
60000/60000 [==============================] - 4s 59us/sample - loss: 0.3319 - acc: 0.8795 - val_loss: 0.3679 - val_acc: 0.8696
Epoch 4/10
60000/60000 [==============================] - 3s 55us/sample - loss: 0.3080 - acc: 0.8870 - val_loss: 0.3596 - val_acc: 0.8676
Epoch 5/10
60000/60000 [==============================] - 3s 54us/sample - loss: 0.2907 - acc: 0.8924 - val_loss: 0.3569 - val_acc: 0.8710
Epoch 6/10
60000/60000 [==============================] - 4s 61us/sample - loss: 0.2781 - acc: 0.8965 - val_loss: 0.3432 - val_acc: 0.8782
Epoch 7/10
60000/60000 [==============================] - 4s 59us/sample - loss: 0.2655 - acc: 0.9019 - val_loss: 0.3315 - val_acc: 0.8800
Epoch 8/10
60000/60000 [==============

In [9]:
# Evaluate the model
test_loss, test_acc = keras_model.evaluate(X_test, y_test)
print("Test accuracy:", test_acc)

Test accuracy: 0.8788


In [ ]:
#now onto deepexplain!

In [10]:
# Load your pre-trained Keras model
keras_model.save("keras_model")
#keras_model.save('./fc-task-decoding')

INFO:tensorflow:Assets written to: keras_model/assets


INFO:tensorflow:Assets written to: keras_model/assets


In [ ]:
keras_model.save('keras_model.h5')  # Save model in HDF5 format
# binary data format that allows you to store the model architecture, weights, and other configuration details in a single file. 

In [4]:
# Load the Keras model from the saved HDF5 file
keras_model = keras.models.load_model('keras_model.h5')

In [5]:
# Create a DeepExplain instance
session = tf.compat.v1.keras.backend.get_session()

In [6]:
# Create a DeepExplain explainer using the Keras session
with DeepExplain(session=session) as de:
    # Load and preprocess the Fashion MNIST dataset
    (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
    X_test = X_test.astype('float32') / 255.0  # Normalize pixel values

    # Assuming you want to explain predictions for the first 10 images in the test set
    num_samples = 10
    input_data = X_test[:num_samples]  # Use the first 10 images as input_data

    # Get the model's output tensor
    output_tensor = keras_model.output
    # Get the model's input tensor
    input_tensor = keras_model.input
    # Explain the model's predictions using DeepExplain
    explanation = de.explain('grad*input', output_tensor, input_tensor, input_data)

RecursionError: maximum recursion depth exceeded while calling a Python object

In [9]:
type(X_test)

numpy.ndarray

In [ ]:
# Now you can use the explanation as needed
print(explanation)